In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from sympy.physics.units import momentum
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False

%matplotlib inline

### 记载历史数据文件

In [15]:
dataset = pd.read_csv('./data/load.csv', index_col=[0])
dataset = dataset.fillna(method='pad')
dataset = np.array(dataset)
# 将所有的数据放到一个列表里面，方便后续的训练集和测试集的制作
a = []
for item in dataset:
    for i in item:
        a.append(i)
frame = pd.DataFrame(a)
frame

,0
0,5687.054687
1,5460.153257
2,5288.463742
3,5078.027286
4,4918.009490
...,...
35035,6798.593207
35036,6553.228700
35037,6321.855764
35038,6151.673338


### 划分训练集和验证集

In [16]:
train = frame.iloc[0:int(len(a)*0.8), [0]] # 序列数据，只能按顺序划分数据集，前80%作为训练集，80%~90%作为验证集
val = frame.iloc[int(len(a)*0.8):int(len(a)*0.9), [0]]
print(len(train),len(val))

28032 3504


### 进行数据归一化，将数据归一化到0-1之间

In [17]:
scaler = MinMaxScaler(feature_range=(0, 1))
train = scaler.fit_transform(train)
val = scaler.fit_transform(val)

### 设置训练集的特征列表和对应标签列表

In [19]:
x_train = []
y_train = []

### 将前96个采样点的负荷特征作为训练特征添加到列表中，按照上述规律不断滑动取值

In [18]:
class  LOAD_MODEL(nn.Module):
    def __init__(self):
        super(LOAD_MODEL, self).__init__()

